# Part A: Newtonian Dynamics 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi, cos, sin
import os

# ----------------------------
# Create output directory
# ----------------------------
output_dir = "Classical Dynamics"
os.makedirs(output_dir, exist_ok=True)


# ---------------------------
# Geometry
# ---------------------------

def is_inside_stadium(x, y, a, b, rx, ry):
    if -a <= x <= a:
        return abs(y) <= b
    elif x > a:
        return ((x - a)**2)/(rx**2) + (y**2)/(ry**2) <= 1
    else:
        return ((x + a)**2)/(rx**2) + (y**2)/(ry**2) <= 1


def boundary_normal(x, y, a, b, rx, ry):
    if abs(y - b) < 1e-8 and -a <= x <= a:
        return np.array([0.0, -1.0], dtype=float)
    if abs(y + b) < 1e-8 and -a <= x <= a:
        return np.array([0.0, 1.0], dtype=float)

    val_right = ((x - a)**2)/(rx**2) + (y**2)/(ry**2)
    if abs(val_right - 1) < 1e-6 and x >= a:
        gx, gy = 2*(x - a)/(rx**2), 2*y/(ry**2)
        nin = -np.array([gx, gy], dtype=float)
        return nin/np.linalg.norm(nin)

    val_left = ((x + a)**2)/(rx**2) + (y**2)/(ry**2)
    if abs(val_left - 1) < 1e-6 and x <= -a:
        gx, gy = 2*(x + a)/(rx**2), 2*y/(ry**2)
        nin = -np.array([gx, gy], dtype=float)
        return nin/np.linalg.norm(nin)

    return np.array([1.0, 0.0], dtype=float)


def reflect_velocity(v, n):
    return v - 2*np.dot(v, n)*n

# ---------------------------
# Integration with reflections
# ---------------------------

def find_collision(x, v, dt, a, b, rx, ry):
    x_end = x + v*dt
    if is_inside_stadium(x_end[0], x_end[1], a, b, rx, ry):
        return False, None, None

    tlo, thi = 0.0, dt
    for _ in range(40):
        tm = 0.5*(tlo + thi)
        pt = x + v*tm
        if is_inside_stadium(pt[0], pt[1], a, b, rx, ry):
            tlo = tm
        else:
            thi = tm
    t_col = 0.5*(tlo + thi)
    return True, t_col, x + v*t_col


def integrate_particle(x0, v0, a, b, rx, ry, dt, T, record_collisions=False):
    nsteps = int(np.ceil(T/dt))
    x, v, t = np.array(x0, dtype=float), np.array(v0, dtype=float), 0.0
    times, pos, vel = [t], [x.copy()], [v.copy()]
    collisions = []

    for _ in range(nsteps):
        remaining = dt
        while remaining > 1e-12:
            collided, tcol, _ = find_collision(x, v, remaining, a, b, rx, ry)
            if not collided:
                x += v*remaining; t += remaining; remaining = 0
            else:
                x += v*tcol; t += tcol; remaining -= tcol
                n = boundary_normal(x[0], x[1], a, b, rx, ry)
                v_new = reflect_velocity(v, n)
                if record_collisions:
                    collisions.append((x.copy(), v.copy(), v_new.copy(), n.copy()))
                v = v_new; x += v*1e-10
        times.append(t); pos.append(x.copy()); vel.append(v.copy())

    if record_collisions:
        return np.array(times), np.array(pos), np.array(vel), collisions
    return np.array(times), np.array(pos), np.array(vel)

# ---------------------------
# Utilities
# ---------------------------

def plot_stadium(a, b, rx, ry, ax):
    xs = np.linspace(-a, a, 200)
    ax.plot(xs, [b]*len(xs), 'k-')
    ax.plot(xs, [-b]*len(xs), 'k-')
    th = np.linspace(-pi/2, pi/2, 200)
    ax.plot(a + rx*np.cos(th), ry*np.sin(th), 'k-')
    ax.plot(-a + rx*np.cos(th + pi), ry*np.sin(th + pi), 'k-')
    ax.set_aspect('equal', 'box')


def kinetic_energy(v):
    return 0.5*np.dot(v, v)


def plot_results(results, a, b, rx, ry, T, title, filename, force_color=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
    plot_stadium(a, b, rx, ry, ax1)
    if force_color is None:
        colors = [f"C{i}" for i in range(len(results))]
    else:
        colors = [force_color for _ in range(len(results))]
    for i, (times, pos, vel) in enumerate(results):
        ax1.plot(pos[:,0], pos[:,1], '-', color=colors[i], label=f"particle {i}")
        ax1.plot(pos[0,0], pos[0,1], 'o', color=colors[i])
        energies = [kinetic_energy(vv) for vv in vel]
        ax2.plot(times, energies, '-', color=colors[i], label=f"particle {i}")

    ax1.set_title(f"Trajectories ({title})")
    ax1.legend()
    ax2.set_xlabel('t'); ax2.set_ylabel('Energy')
    ax2.set_title('Kinetic Energy vs Time')
    ax2.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename), dpi=600)
    plt.close()

# ---------------------------
# Experiments
# ---------------------------
if __name__ == "__main__":
    a, b = 1.0, 0.5
    rx, ry = 0.6, b
    dt = 0.001

    np.random.seed(12345)

    # 1. Single trajectory with normals
    x0, v0 = [0,0], [0.7, 0.3]
    times, pos, vel, colls = integrate_particle(x0, v0, a,b,rx,ry, dt, 5, True)
    fig, ax = plt.subplots(figsize=(7,5))
    plot_stadium(a,b,rx,ry,ax)
    ax.plot(pos[:,0], pos[:,1], label="trajectory")
    for (p, vin, vout, n) in colls:
        ax.plot(p[0], p[1], 'ro')
        ax.arrow(p[0], p[1], 0.15*n[0], 0.15*n[1], head_width=0.03, fc='g')
    ax.set_title("Trajectory with Normals")
    ax.legend()
    plt.savefig(os.path.join(output_dir, "trajectory_with_normals.png"), dpi=600)
    plt.close()
    # Sanity Check: Verify Law of Reflection
    def angle_between(v1, v2):
        """Calculates the angle in radians between two vectors."""
        v1_u = v1 / np.linalg.norm(v1)
        v2_u = v2 / np.linalg.norm(v2)
        # Clip to handle potential floating point errors outside [-1, 1]
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

    print("\n--- Verifying Law of Reflection (Test 1) ---")
    for i, (p, v_in, v_out, n) in enumerate(colls):
        # Angle of incidence is between the normal and the *negative* incoming velocity
        angle_in = angle_between(-v_in, n)
        # Angle of reflection is between the normal and the outgoing velocity
        angle_out = angle_between(v_out, n)
        print(f"Collision {i}: Angle In = {np.degrees(angle_in):.2f}°, Angle Out = {np.degrees(angle_out):.2f}°")
        # Assert that the angles are equal to within a small tolerance
        assert np.isclose(angle_in, angle_out), "Law of reflection violated!"
    print("--- Sanity check passed: Angles match. ---\n")
    
    # 2. Multiple random initial positions + velocities
    results = []
    for _ in range(5):
        pos0 = [np.random.uniform(-a,a), np.random.uniform(-b,b)]
        while not is_inside_stadium(pos0[0], pos0[1], a,b,rx,ry):
            pos0 = [np.random.uniform(-a,a), np.random.uniform(-b,b)]
        theta, speed = np.random.uniform(0,2*pi), np.random.uniform(0.1,1)
        v0 = [speed*cos(theta), speed*sin(theta)]
        results.append(integrate_particle(pos0, v0, a,b,rx,ry, dt, 10))
    plot_results(results, a,b,rx,ry,10, "Random Initial Conditions", "random_conditions.png")

    # 3. Chaos: same velocity, nearby points
    theta, speed = np.random.uniform(0,2*pi), 1.0
    v0 = [speed*cos(theta), speed*sin(theta)]
    results = []
    for _ in range(4):
        dx, dy = np.random.uniform(-0.2,0.2), np.random.uniform(-0.2,0.2)
        pos0 = [0+dx,0+dy]
        if is_inside_stadium(pos0[0], pos0[1], a,b,rx,ry):
            results.append(integrate_particle(pos0, v0, a,b,rx,ry, dt, 10))
    plot_results(results, a,b,rx,ry,10, "Chaos Test", "chaos.png")

    # 4. Ergodicity: one trajectory for different times
    pos0 = [1.2, -0.3]; v0 = [cos(theta), sin(theta)]
    for T in [5, 50, 500]:
        res = [integrate_particle(pos0, v0, a,b,rx,ry, dt, T)]
        plot_results(res, a,b,rx,ry,T, f"Ergodicity_2_T={T}", f"ergodicity_2_T{T}.png", force_color="orange")
        

# Part B: Quantum dynamics

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.sparse import identity, lil_matrix
from scipy.sparse.linalg import factorized
from tqdm import tqdm
import time
import matplotlib.patches as mpatches

# ----------------------------
# Create main output directory
# ----------------------------
output_dir = "Quantum Dynamics"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created main directory: {output_dir}")
else:
    print(f"Main directory already exists: {output_dir}")

# ----------------------------
# Initial conditions list
# ----------------------------
initial_conditions = [
    (-15.0, -5.0, 3.0, 5.0, 1.5),
    (-10.0, 0.0, 2.0, 4.0, 1.2),
    (-5.0, 5.0, 4.0, 3.0, 1.0),
]

# ----------------------------
# Stadium parameters
# ----------------------------
a, b, rx, ry = 20.0, 10.0, 5.0, 10.0
Nx, Ny = 600, 600
x = np.linspace(-a-rx, a+rx, Nx)
y = np.linspace(-b-ry, b+ry, Ny)
dx, dy = x[1]-x[0], y[1]-y[0]
X, Y = np.meshgrid(x, y, indexing='ij')

# ----------------------------
# Mask and index mapping
# ----------------------------
def is_inside_stadium(x, y, a, b, rx, ry):
    if -a <= x <= a:
        return abs(y) <= b
    elif x > a:
        return ((x - a)**2)/(rx**2) + (y**2)/(ry**2) <= 1.0
    else:
        return ((x + a)**2)/(rx**2) + (y**2)/(ry**2) <= 1.0

mask = np.vectorize(is_inside_stadium)(X, Y, a, b, rx, ry)
mask_flat = mask.flatten()
inside_idx = np.where(mask_flat)[0]
N_inside = len(inside_idx)
idx_map = -np.ones(mask_flat.shape, dtype=int)
idx_map[inside_idx] = np.arange(N_inside)

# ----------------------------
# Gaussian wavepacket
# ----------------------------
def gaussian_wavepacket(X, Y, x0, y0, px0, py0, sigma):
    # Set hbar=1
    return (1/(sigma*np.sqrt(np.pi))) * np.exp(-((X-x0)**2 + (Y-y0)**2)/(2*sigma**2)) * np.exp(1j*(px0*X + py0*Y))

# ----------------------------
# Laplacian matrix
# ----------------------------
t0 = time.time()
from scipy.sparse import csc_matrix
L = lil_matrix((N_inside, N_inside), dtype=float)
c_center = -2.0/dx**2 - 2.0/dy**2
cx = 1.0/dx**2
cy = 1.0/dy**2
for i in range(Nx):
    base = i*Ny
    for j in range(Ny):
        global_idx = base+j
        if not mask[i, j]: continue
        row = idx_map[global_idx]; L[row, row] = c_center
        if i-1>=0 and mask[i-1,j]: L[row, idx_map[(i-1)*Ny+j]] = cx
        if i+1<Nx and mask[i+1,j]: L[row, idx_map[(i+1)*Ny+j]] = cx
        if j-1>=0 and mask[i,j-1]: L[row, idx_map[i*Ny+(j-1)]] = cy
        if j+1<Ny and mask[i,j+1]: L[row, idx_map[i*Ny+(j+1)]] = cy
L = L.tocsc()


print(f"Laplacian built in {time.time()-t0:.2f}s, nnz={L.nnz}")

# ----------------------------
# Crank-Nicolson operator (with H = -1/2 * L, assuming m=1, hbar=1)
# ----------------------------
dt = 0.05
I_inside = identity(N_inside, dtype=complex)
A = (I_inside - 0.25j*dt*L).tocsc()
B = (I_inside + 0.25j*dt*L).tocsc()
solve = factorized(A)
nsteps, frame_skip = 800, 6

# ----------------------------
# Boundary & classical functions
# ----------------------------
def boundary_normal(x, y, a, b, rx, ry):
    eps = 1e-9
    if abs(abs(y)-b) < 1e-8 and -a-1e-8 <= x <= a+1e-8: 
        return np.array([0.0, -1.0]) if y>0 else np.array([0.0, 1.0])
    val_right = ((x - a)**2)/(rx**2) + (y**2)/(ry**2)
    if abs(val_right-1.0)<1e-6 and x>=a-1e-6:
        gx, gy = 2.0*(x-a)/rx**2, 2.0*y/ry**2
        nin=-np.array([gx,gy]); nrm=np.linalg.norm(nin); return nin/max(nrm,eps)
    val_left = ((x + a)**2)/(rx**2) + (y**2)/(ry**2)
    if abs(val_left-1.0)<1e-6 and x<=-a+1e-6:
        gx, gy = 2.0*(x+a)/rx**2, 2.0*y/ry**2
        nin=-np.array([gx,gy]); nrm=np.linalg.norm(nin); return nin/max(nrm,eps)
    dtop, dbot, dright, dleft = abs(y-b), abs(y+b), abs(val_right-1.0), abs(val_left-1.0)
    mind = min(dtop, dbot, dright, dleft)
    if mind==dtop: return np.array([0.0,-1.0])
    if mind==dbot: return np.array([0.0,1.0])
    if mind==dright: gx, gy=2.0*(x-a)/rx**2,2.0*y/ry**2; nin=-np.array([gx,gy]); return nin/np.linalg.norm(nin)
    gx, gy=2.0*(x+a)/rx**2,2.0*y/ry**2; nin=-np.array([gx,gy]); return nin/np.linalg.norm(nin)

def reflect_velocity(v, n_inward):
    return v - 2.0 * np.dot(v, n_inward) * n_inward

def find_collision_time_between(x, v, dt, a, b, rx, ry):
    x_end = x + v*dt
    if is_inside_stadium(x_end[0], x_end[1], a, b, rx, ry): return False, None, None
    t_lo, t_hi = 0.0, dt
    def inside_at(t): return is_inside_stadium(*(x+v*t), a, b, rx, ry)
    if not inside_at(t_lo): return True, 0.0, x.copy()
    if inside_at(t_hi): return False, None, None
    for _ in range(50):
        tm = 0.5*(t_lo+t_hi)
        if inside_at(tm): t_lo=tm
        else: t_hi=tm
    t_collision = 0.5*(t_lo+t_hi)
    return True, t_collision, x + v*t_collision

def integrate_particle(x0, v0, a, b, rx, ry, dt, T):
    nsteps=int(np.ceil(T/dt)); t,x,v=0.0,x0.copy(),v0.copy()
    times, positions, velocities=[0.0],[x.copy()],[v.copy()]
    for step in range(nsteps):
        remaining_dt, subiter = dt, 0
        while remaining_dt>1e-12 and subiter<20:
            subiter+=1
            collided,tcol,_ = find_collision_time_between(x,v,remaining_dt,a,b,rx,ry)
            if not collided: x+=v*remaining_dt; t+=remaining_dt; remaining_dt=0.0; break
            else:
                if tcol<1e-14: n=boundary_normal(x[0],x[1],a,b,rx,ry); v=reflect_velocity(v,n); x+=v*1e-10; continue
                x+=v*tcol; t+=tcol; remaining_dt-=tcol; n=boundary_normal(x[0],x[1],a,b,rx,ry); v=reflect_velocity(v,n)
        times.append(t); positions.append(x.copy()); velocities.append(v.copy())
    return np.array(times), np.array(positions), np.array(velocities)

# ----------------------------
# Loop over runs
# ----------------------------
for run_idx, (x0, y0, px0, py0, sigma) in enumerate(initial_conditions, start=1):
    run_label = f"Run {run_idx}"
    output_subdir = os.path.join(output_dir, f"Run_{run_idx}")
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)
        print(f"Created subdirectory: {output_subdir}")
    else:
        print(f"Subdirectory already exists: {output_subdir}")

    # Initial wavepacket
    psi0 = gaussian_wavepacket(X, Y, x0, y0, px0, py0, sigma)
    psi0[~mask] = 0.0
    norm0 = np.sqrt(np.sum(np.abs(psi0)**2)*dx*dy)
    psi0/=norm0
    psi_inside = psi0.flatten()[inside_idx].astype(complex)

    # Time evolution
    frames, times_list, norms = [], [], []
    expect_x, expect_y = [], []

    for step in tqdm(range(nsteps), desc=f"Simulating {run_label}"):
        psi_inside = solve(B.dot(psi_inside))
        psi_full = np.zeros(mask_flat.shape, dtype=complex)
        psi_full[inside_idx]=psi_inside
        psi_2d = psi_full.reshape((Nx, Ny))
        
        prob_density=np.abs(psi_2d)**2
        norm = np.sum(prob_density)*dx*dy # Norm is sum, not sqrt of sum
        norms.append(norm)

        # Normalize norm to 1 to prevent numerical drift in expectation values
        norm_factor = norm if norm > 1e-9 else 1.0

        expect_x.append(np.sum(prob_density*X)*dx*dy / norm_factor)
        expect_y.append(np.sum(prob_density*Y)*dx*dy / norm_factor)
        times_list.append(step*dt)

        if step % frame_skip == 0: frames.append(prob_density.copy())

    expect_x = np.array(expect_x); expect_y = np.array(expect_y)
    

    # Classical trajectory
    x0_classical = np.array([x0,y0],float)
    v0_classical = np.array([px0,py0],float) # m=1
    T_total = nsteps*dt
    classical_times, classical_positions, classical_velocities = integrate_particle(
        x0_classical, v0_classical, a,b,rx,ry,dt,T_total)

    # ----------------------------
    # Save static plots
    # ----------------------------
    # Trajectories
    plt.figure(figsize=(8,8))
    plt.contour(X, Y, mask.astype(int), levels=[0.5], colors='k', linewidths=1.5)
    plt.plot(expect_x, expect_y, '-', color='red', lw=2.5, label=r'Quantum $\langle \mathbf{r} \rangle$')
    plt.plot(classical_positions[:,0], classical_positions[:,1], '--', color='blue', lw=1.8, label='Classical trajectory')
    plt.scatter([classical_positions[0,0]], [classical_positions[0,1]], s=80, marker='s', color='blue', edgecolor='k', label='Start Point')
    plt.scatter([classical_positions[-1,0]], [classical_positions[-1,1]], s=80, marker='D', color='blue', edgecolor='k', label='Classical End Point')
    plt.scatter([expect_x[-1]], [expect_y[-1]], s=80, marker='X', color='red', edgecolor='k', label='Quantum End Point')
    plt.gca().set_aspect('equal', 'box')
    plt.xlabel('x'); plt.ylabel('y'); plt.title(f'Comparison of Trajectories for {run_label}')
    plt.legend()
    plt.tight_layout()
    trajectory_file = os.path.join(output_subdir, f"stadium_trajectory_comparison_{run_label.replace(' ','_')}.png")
    plt.savefig(trajectory_file)
    plt.close()
    print(f"Saved trajectory plot: {trajectory_file}")

    # Norm
    plt.figure(figsize=(8,4)); plt.plot(np.arange(nsteps)*dt, norms, 'b-', lw=2)
    plt.xlabel("Time"); plt.ylabel(r"$||\psi||^2$"); plt.title(f"Quantum Norm vs Time ({run_label})"); plt.grid(alpha=0.3)
    plt.tight_layout()
    norm_file = os.path.join(output_subdir, f"norm_vs_time_{run_label.replace(' ','_')}.png")
    plt.savefig(norm_file)
    plt.close()
    print(f"Saved norm plot: {norm_file}")

    # Classical speed
    speed_classical = np.linalg.norm(classical_velocities, axis=1)
    plt.figure(figsize=(8,4)); plt.plot(classical_times, speed_classical, 'r-', lw=2)
    plt.xlabel("Time"); plt.ylabel("$|v|$"); plt.title(f"Classical Speed vs Time ({run_label})"); plt.grid(alpha=0.3)
    plt.tight_layout()
    speed_file = os.path.join(output_subdir, f"classical_speed_{run_label.replace(' ','_')}.png")
    plt.savefig(speed_file)
    plt.close()
    print(f"Saved classical speed plot: {speed_file}")


    # ----------------------------
    # Animation
    # ----------------------------
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    vmax = np.max(frames[0]) if len(frames) > 0 and np.max(frames[0]) > 0 else 1.0
    im = ax1.imshow(frames[0].T, extent=[x[0],x[-1],y[0],y[-1]], origin='lower', cmap='hot', vmin=0, vmax=vmax, aspect='auto')
    ax1.contour(X.T, Y.T, mask.T, levels=[0.5], colors='white', linewidths=1.2, alpha=0.9)
    ax1.set_title('Probability density'); ax1.set_xlabel('x'); ax1.set_ylabel('y')
    cbar = fig.colorbar(im, ax=ax1, fraction=0.046, pad=0.04)
    cbar.set_label(r'Probability density $|\psi|^2$', rotation=90, labelpad=10)

    line_quantum, = ax2.plot([], [], '-', color='red', lw=2.5, label=r'Quantum $\langle \mathbf{r} \rangle$')
    line_classical, = ax2.plot([], [], '--', color='blue', lw=1.8, label='Classical trajectory')
    patch_prob = mpatches.Patch(color='gray', alpha=0.5, label=r'Probability density $|\psi|^2$')

    ax2.scatter([classical_positions[0,0]], [classical_positions[0,1]], s=70, marker='s', color='blue', edgecolor='k', zorder=6, label='Start Point')
    ax2.contour(X.T, Y.T, mask.T, levels=[0.5], colors='k', linewidths=1.2)

    ax2.set_xlim(x[0], x[-1]); ax2.set_ylim(y[0], y[-1])
    ax2.set_xlabel('x'); ax2.set_ylabel('y'); ax2.set_title(f'Evolution of Trajectories in {run_label}')
    ax2.legend(handles=[patch_prob, line_quantum, line_classical], loc='upper right')
    ax2.set_aspect('equal', 'box')

    n_frames = len(frames)
    def animate(frame_idx):
        prob = frames[frame_idx]
        im.set_array(prob.T)
        im.set_clim(0, prob.max() if prob.max()>0 else vmax)
        step_idx = min(frame_idx*frame_skip, len(expect_x)-1)
        line_quantum.set_data(expect_x[:step_idx+1], expect_y[:step_idx+1])
        current_time = min(step_idx*dt, classical_times[-1])
        mask_c = classical_times <= current_time
        if np.any(mask_c):
            line_classical.set_data(classical_positions[mask_c][:,0], classical_positions[mask_c][:,1])
        else:
            line_classical.set_data([], [])
        return im, line_quantum, line_classical

    ani = FuncAnimation(fig, animate, frames=n_frames, interval=80, blit=False)
    out_filename = os.path.join(output_subdir, f"quantum_classical_stadium_{run_label.replace(' ','_')}.mp4")
    from matplotlib import animation
    try:
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=12, metadata=dict(artist='CN-solver'), bitrate=2500)
        ani.save(out_filename, writer=writer, dpi=200)
        print(f"Saved animation MP4: {out_filename}")
    except Exception as e:
        print(f"FFmpeg failed: {e}, saving GIF instead")
        out_filename = os.path.join(output_subdir, f"quantum_classical_stadium_{run_label.replace(' ','_')}.gif")
        ani.save(out_filename, writer='pillow', fps=8)
        print(f"Saved animation GIF: {out_filename}")
    plt.close(fig)

    print(f"Finished all outputs for {run_label}\n")

Main directory already exists: Quantum Dynamics
Laplacian built in 1.91s, nnz=857764
Subdirectory already exists: Quantum Dynamics/Run_1


Simulating Run 1: 100%|██████████| 800/800 [00:40<00:00, 19.68it/s]


Saved trajectory plot: Quantum Dynamics/Run_1/stadium_trajectory_comparison_Run_1.png
Saved norm plot: Quantum Dynamics/Run_1/norm_vs_time_Run_1.png
Saved classical speed plot: Quantum Dynamics/Run_1/classical_speed_Run_1.png
Saved animation MP4: Quantum Dynamics/Run_1/quantum_classical_stadium_Run_1.mp4
Finished all outputs for Run 1

Subdirectory already exists: Quantum Dynamics/Run_2


Simulating Run 2: 100%|██████████| 800/800 [00:44<00:00, 18.13it/s]


Saved trajectory plot: Quantum Dynamics/Run_2/stadium_trajectory_comparison_Run_2.png
Saved norm plot: Quantum Dynamics/Run_2/norm_vs_time_Run_2.png
Saved classical speed plot: Quantum Dynamics/Run_2/classical_speed_Run_2.png
Saved animation MP4: Quantum Dynamics/Run_2/quantum_classical_stadium_Run_2.mp4
Finished all outputs for Run 2

Subdirectory already exists: Quantum Dynamics/Run_3


Simulating Run 3: 100%|██████████| 800/800 [00:45<00:00, 17.70it/s]


Saved trajectory plot: Quantum Dynamics/Run_3/stadium_trajectory_comparison_Run_3.png
Saved norm plot: Quantum Dynamics/Run_3/norm_vs_time_Run_3.png
Saved classical speed plot: Quantum Dynamics/Run_3/classical_speed_Run_3.png
Saved animation MP4: Quantum Dynamics/Run_3/quantum_classical_stadium_Run_3.mp4
Finished all outputs for Run 3



# Part C: Loschimdt Echo

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import identity, lil_matrix
from tqdm import tqdm
import time

# ----------------------------
# Create main output directory
# ----------------------------
output_dir = "Loschmidt_Echo_Magnitude_Study"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created main directory: {output_dir}")
else:
    print(f"Main directory already exists: {output_dir}")

# ----------------------------
# --- Perturbation Magnitudes ---
# ----------------------------
da_values = [0.2, 0.4, 1.0, 4.0]
drx_values = [0.05, 0.1, 0.25, 0.5]
db_values = [0.1, 0.2, 0.5, 2]
print(f"Perturbation strengths for 'a': {da_values}")
print(f"Perturbation strengths for 'rx': {drx_values}")
print(f"Perturbation strengths for 'b'/'ry': {db_values}")

# ----------------------------
# Initial conditions list (using one for this study)
# ----------------------------
#[(x0, y0, px0, py0, sigma), ...]
initial_conditions = [
    (-15.0, -5.0, 3.0, 5.0, 1.5),
    (-10.0, 0.0, 2.0, -4.0, 1.2),
] # You can add more initial conditions if desired

# ----------------------------
# Base Stadium parameters
# ----------------------------
a_base, b_base, rx_base, ry_base = 20.0, 10.0, 5.0, 10.0
Nx, Ny = 600, 600
# Make grid large enough to contain all perturbed stadiums
x_max = a_base + 2*max(drx_values) + 2*max(da_values) + 2 # Add some buffer
y_max = b_base + max(db_values) + 2 # Add some buffer
x = np.linspace(-x_max, x_max, Nx)
y = np.linspace(-y_max, y_max, Ny)
dx, dy = x[1]-x[0], y[1]-y[0]
X, Y = np.meshgrid(x, y, indexing='ij')

# ----------------------------
# Helper functions
# ----------------------------
def is_inside_stadium(x, y, a, b, rx, ry):
    """Checks if a point (x,y) is inside the stadium geometry."""
    if -a <= x <= a:
        return abs(y) <= b
    elif x > a:
        return ((x - a)**2)/(rx**2) + (y**2)/(ry**2) <= 1.0
    else: # x < -a
        return ((x + a)**2)/(rx**2) + (y**2)/(ry**2) <= 1.0

def build_laplacian_system(mask, Nx, Ny, dx, dy):
    """Builds the sparse Laplacian matrix and corresponding index maps."""
    mask_flat = mask.flatten()
    inside_idx = np.where(mask_flat)[0]
    N_inside = len(inside_idx)
    
    t0 = time.time()
    L = lil_matrix((N_inside, N_inside), dtype=float)
    c_center, cx, cy = -2.0/dx**2 - 2.0/dy**2, 1.0/dx**2, 1.0/dy**2
    idx_map = -np.ones(mask_flat.shape, dtype=int)
    idx_map[inside_idx] = np.arange(N_inside)

    for i in range(Nx):
        for j in range(Ny):
            if not mask[i, j]: continue
            row = idx_map[i*Ny + j]
            L[row, row] = c_center
            if i > 0 and mask[i-1, j]: L[row, idx_map[(i-1)*Ny + j]] = cx
            if i < Nx-1 and mask[i+1, j]: L[row, idx_map[(i+1)*Ny + j]] = cx
            if j > 0 and mask[i, j-1]: L[row, idx_map[i*Ny + (j-1)]] = cy
            if j < Ny-1 and mask[i, j+1]: L[row, idx_map[i*Ny + (j+1)]] = cy

    L_csc = L.tocsc()
    print(f"  Laplacian built in {time.time()-t0:.2f}s, nnz={L_csc.nnz}")
    return L_csc, inside_idx, N_inside

def get_crank_nicolson_solver(L, N_inside, dt):
    """Creates the factorized solver for the Crank-Nicolson method."""
    I = identity(N_inside, dtype=complex)
    A = (I - 0.25j*dt*L).tocsc() # H = -1/2 L
    B = (I + 0.25j*dt*L).tocsc()
    return factorized(A), B

def gaussian_wavepacket(X, Y, x0, y0, px0, py0, sigma):
    """Creates a 2D Gaussian wavepacket."""
    return (1/(sigma*np.sqrt(np.pi))) * np.exp(-((X-x0)**2 + (Y-y0)**2)/(2*sigma**2)) * np.exp(1j*(px0*X + py0*Y))

def project_and_normalize(psi_2d, mask, inside_idx):
    """Projects the wavepacket onto the stadium grid and normalizes it."""
    psi_proj = psi_2d.copy()
    psi_proj[~mask] = 0.0
    norm = np.sqrt(np.sum(np.abs(psi_proj)**2) * dx * dy)
    psi_proj /= norm
    return psi_proj.flatten()[inside_idx].astype(complex)

# ----------------------------
# Core Simulation Function
# ----------------------------
def run_perturbation_study(pert_type, pert_values, unperturbed_system, psi0_2d, run_label, output_subdir):
    """Runs a full simulation for a set of perturbation values of a single type."""
    
    print(f"\n--- Starting Perturbation Study for '{pert_type}' with values {pert_values} ---")
    
    # Unpack unperturbed system
    mask_unperturbed, idx_unperturbed, solve_unperturbed, B_unperturbed = unperturbed_system

    # Build all perturbed systems for this study
    perturbed_systems = []
    for val in pert_values:
        print(f"Building perturbed system for {pert_type} = {val}...")
        if pert_type == 'a':
            mask_p = np.vectorize(is_inside_stadium)(X, Y, a_base + val, b_base, rx_base, ry_base)
        elif pert_type == 'rx':
            mask_p = np.vectorize(is_inside_stadium)(X, Y, a_base, b_base, rx_base + val, ry_base)
        elif pert_type == 'b':
            mask_p = np.vectorize(is_inside_stadium)(X, Y, a_base, b_base + val, rx_base, ry_base + val)
        
        L_p, idx_p, N_p = build_laplacian_system(mask_p, Nx, Ny, dx, dy)
        solve_p, B_p = get_crank_nicolson_solver(L_p, N_p, dt)
        perturbed_systems.append({'mask': mask_p, 'idx': idx_p, 'solve': solve_p, 'B': B_p})

    # Initialize wavepackets
    psi_unperturbed = project_and_normalize(psi0_2d, mask_unperturbed, idx_unperturbed)
    psi_perturbed_list = [
        project_and_normalize(psi0_2d, sys['mask'], sys['idx']) for sys in perturbed_systems
    ]

    # Time evolution
    times = []
    echo_histories = [[] for _ in pert_values]

    for step in tqdm(range(nsteps), desc=f"Simulating {pert_type} perturbations for {run_label}"):
        psi_unperturbed = solve_unperturbed(B_unperturbed.dot(psi_unperturbed))
        
        psi_2d_unperturbed = np.zeros(Nx * Ny, dtype=complex)
        psi_2d_unperturbed[idx_unperturbed] = psi_unperturbed
        
        for i, sys in enumerate(perturbed_systems):
            psi_perturbed_list[i] = sys['solve'](sys['B'].dot(psi_perturbed_list[i]))
            
            psi_2d_pert = np.zeros(Nx * Ny, dtype=complex)
            psi_2d_pert[sys['idx']] = psi_perturbed_list[i]
            
            overlap = np.sum(np.conj(psi_2d_pert) * psi_2d_unperturbed) * dx * dy
            echo_histories[i].append(np.abs(overlap)**2)
        
        times.append(step * dt)

    # Plotting
    plt.figure(figsize=(12, 7))
    for i, val in enumerate(pert_values):
        label_map = {'a': r'\delta a', 'rx': r'\delta r_x', 'b': r'\delta b'}
        plt.plot(times, echo_histories[i], label=f'${label_map[pert_type]} = {val}$', lw=2.5)
    
    plt.xlabel("Time (t)", fontsize=14)
    plt.ylabel("Loschmidt Echo $L(t)$", fontsize=14)
    plt.title(f"Echo Sensitivity to Perturbation of '{pert_type}' ({run_label})", fontsize=16)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.ylim(0, 1.05)
    plt.legend(fontsize=12)
    plt.tight_layout()
    
    plot_file = os.path.join(output_subdir, f"echo_study_{pert_type}_{run_label}.png")
    plt.savefig(plot_file, dpi=200)
    plt.close()
    print(f"Saved plot: {plot_file}")

# ----------------------------
# Main Execution Loop
# ----------------------------
dt = 0.05
nsteps = 800

for run_idx, (x0, y0, px0, py0, sigma) in enumerate(initial_conditions, start=1):
    run_label = f"Run_{run_idx}"
    output_subdir = os.path.join(output_dir, run_label)
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)
    
    print(f"\n{'='*60}\nSTARTING SIMULATIONS FOR {run_label.upper()}\n{'='*60}")
    
    # --- Build the single Unperturbed System for this run ---
    print("Building Unperturbed Base System...")
    mask_unperturbed = np.vectorize(is_inside_stadium)(X, Y, a_base, b_base, rx_base, ry_base)
    L_unperturbed, idx_unperturbed, N_unperturbed = build_laplacian_system(mask_unperturbed, Nx, Ny, dx, dy)
    solve_unperturbed, B_unperturbed = get_crank_nicolson_solver(L_unperturbed, N_unperturbed, dt)
    unperturbed_system = (mask_unperturbed, idx_unperturbed, solve_unperturbed, B_unperturbed)
    
    # --- Create the initial wavepacket ---
    psi0_2d = gaussian_wavepacket(X, Y, x0, y0, px0, py0, sigma)
    
    # --- Run the three separate studies ---
    run_perturbation_study('a', da_values, unperturbed_system, psi0_2d, run_label, output_subdir)
    run_perturbation_study('rx', drx_values, unperturbed_system, psi0_2d, run_label, output_subdir)
    run_perturbation_study('b', db_values, unperturbed_system, psi0_2d, run_label, output_subdir)

print(f"\n\n{'='*60}\nAll simulations and analysis complete.\n{'='*60}")


Main directory already exists: Loschmidt_Echo
Perturbation strengths for 'a': [0.2]
Perturbation strengths for 'rx': [0.05]
Perturbation strengths for 'b'/'ry': [0.1]

STARTING SIMULATIONS FOR RUN_1
Building Unperturbed Base System...
  Laplacian built in 2.66s, nnz=1477968

--- Starting Perturbation Study for 'rx' with values [0.05] ---
Building perturbed system for rx = 0.05...
  Laplacian built in 2.62s, nnz=1478148


Simulating rx perturbations for Run_1: 100%|██████████| 1600/1600 [04:41<00:00,  5.68it/s]


Saved plot: Loschmidt_Echo/Run_1/echo_study_rx_Run_1.png

STARTING SIMULATIONS FOR RUN_2
Building Unperturbed Base System...
  Laplacian built in 3.05s, nnz=1477968

--- Starting Perturbation Study for 'rx' with values [0.05] ---
Building perturbed system for rx = 0.05...
  Laplacian built in 2.63s, nnz=1478148


Simulating rx perturbations for Run_2: 100%|██████████| 1600/1600 [04:54<00:00,  5.42it/s]


Saved plot: Loschmidt_Echo/Run_2/echo_study_rx_Run_2.png


All simulations and analysis complete.
